In [ ]:
import os
import re
from datetime import datetime
from itertools import islice
from string import ascii_uppercase

import googlemaps
import gspread
import pandas as pd
import pytz
from oauth2client.service_account import ServiceAccountCredentials
from pytz import timezone

### The Goal

Retrieve up-to-date travel time data for the most efficient route between a series of cities, transform it, and send it to Google Sheets to feed a Tableau workbook.

In [ ]:
gmaps = googlemaps.Client(key=os.getenv('GMAPS_DISTANCE_KEY'))

In [ ]:
origins = ['Portland, OR, USA',
           'Lincoln City, OR, USA',
           'Oregon City, OR, USA',
           'Boise, ID, USA']
destinations = ['Salem, OR, USA',
                'Corvallis, OR, USA',
                'Madras, OR, USA',
                'Baker City, OR, USA',
               'Lincoln City, OR, USA']

In [ ]:
results = gmaps.distance_matrix(origins=origins,
                                destinations=destinations,
                                units='imperial',
                                departure_time='now',
                                traffic_model='best_guess')

#### Write a function to parse the results json for easy conversion to a pandas dataframe

In [ ]:
new_rows = []
for i, origin in enumerate(origins):
    for j, destination in enumerate(destinations):
        fun_dict = {'origin':origin,
                   'destination': destination}
        intermediate_thing = results['rows'][i]['elements'][j]
        for key,value in intermediate_thing.items():
            if (key == 'status'):
                continue
            fun_dict.update({key: value['value']})
        new_rows.append(fun_dict)

In [ ]:
df = pd.DataFrame(new_rows)

In [ ]:
df.head()

#### Perform calculations to more intuitively display time information (returned by the API in seconds)

In [ ]:
df['duration formatted'] = round(df['duration']/3600,1)

In [ ]:
df['traffic_duration formatted'] = round(df['duration_in_traffic']/3600,1)

In [ ]:
df['difference'] = round(df['duration_in_traffic']/df['duration'],2)

#### Simplify city names

In [ ]:
df['destination'] = df['destination'].apply(lambda x: x.split(",")[0])
df['origin'] = df['origin'].apply(lambda x: x.split(",")[0])

In [ ]:
df = df.drop([6,7,8,9,11,12,13,14,15,16,17,19])

In [ ]:
final_df = df[['destination','origin','duration formatted',
               'traffic_duration formatted','difference']]

In [ ]:
final_df.rename(columns={
        'destination':'To',
        'origin':'From',
        'duration formatted':'Typical Travel Time (Hours)',
        'traffic_duration formatted':'Current Travel Time (Hours)',
        'difference':'Difference'
    },inplace=True)

In [ ]:
scope = ['https://spreadsheets.google.com/feeds']
credentials = ServiceAccountCredentials.from_json_keyfile_name('client_secret.json', scope)
gc = gspread.authorize(credentials)

In [ ]:
wks = gc.open("eclipse_traffic").sheet1

#### The following cell is code adapted from the df2gspread library, which could not load my credentials for reasons I've yet to debug!

In [ ]:
def grouper(n, iterable):
    it = iter(iterable)
    while True:
        chunk = tuple(islice(it, n))
        if not chunk:
            return
        yield chunk

In [ ]:
# adapted from the df2gspread library, which could not load my credentials
def upload(df, wks, chunk_size=1000,
           col_names=True, row_names=True, clean=True, credentials=None,
           start_cell = 'A1', df_size = False, new_sheet_dimensions = (1000,100)):
    '''
        Upload given Pandas DataFrame to Google Drive and returns 
        gspread Worksheet object

        :param df: Pandas DataFrame
        :param gfile: path to Google Spreadsheet or gspread ID
        :param wks_name: worksheet name
        :param chunk_size: size of chunk to upload
        :param col_names: passing top row to column names for Pandas DataFrame
        :param row_names: passing left column to row names for Pandas DataFrame
        :param clean: clean all data in worksheet before uploading 
        :param credentials: provide own credentials
        :param start_cell: specify where to insert the DataFrame; default is A1
        :param df_size: 
            -If True and worksheet name does NOT exist, will create 
            a new worksheet that is the size of the df; otherwise, by default, 
            creates sheet of 1000x100 cells. 
            -If True and worksheet does exist, will resize larger or smaller to 
            fit new dataframe. 
            -If False and dataframe is larger than existing sheet, will resize 
            the sheet larger.
            -If False and dataframe is smaller than existing sheet, does not resize.
        :param new_sheet_dimensions: tuple of (row, cols) for size of a new sheet
        :type df: class 'pandas.core.frame.DataFrame'
        :type gfile: str
        :type wks_name: str
        :type chunk_size: int
        :type col_names: bool
        :type row_names: bool
        :type clean: bool
        :type credentials: class 'oauth2client.client.OAuth2Credentials'
        :type start_cell: str
        :type df_size: bool
        :type new_sheet_dimensions: tuple
        :returns: gspread Worksheet
        :rtype: class 'gspread.models.Worksheet'

        :Example:

            >>> from df2gspread import df2gspread as d2g
            >>> import pandas as pd
            >>> df = pd.DataFrame([1 2 3])
            >>> wks = d2g.upload(df, wks_name='Example worksheet')
            >>> wks.title
            'Example worksheet'
    '''
    # access credentials
    scope = ['https://spreadsheets.google.com/feeds']
    credentials = ServiceAccountCredentials.from_json_keyfile_name('client_secret.json', scope)
    # auth for gspread
    gc = gspread.authorize(credentials)

    start_col = re.split('(\d+)',start_cell)[0].upper()
    start_row = re.split('(\d+)',start_cell)[1]
    start_row_int, start_col_int = wks.get_int_addr(start_cell)

    # find last index and column name (A B ... Z AA AB ... AZ BA)
    num_rows = len(df.index) + 1 if col_names else len(df.index)
    last_idx_adjust = start_row_int - 1
    last_idx = num_rows + last_idx_adjust

    num_cols = len(df.columns) + 1 if row_names else len(df.columns)
    last_col_adjust = start_col_int - 1
    last_col_int = num_cols + last_col_adjust
    last_col = re.split('(\d+)',(wks.get_addr_int(1, last_col_int)))[0].upper()

    # If user requested to resize sheet to fit dataframe, go ahead and 
    # resize larger or smaller to better match new size of pandas dataframe.
    # Otherwise, leave it the same size unless the sheet needs to be expanded
    # to accomodate a larger dataframe.
    if df_size:
        wks.resize(rows=len(df.index) + col_names, cols=len(df.columns) + row_names)
    if len(df.index) + col_names + last_idx_adjust > wks.row_count:
        wks.add_rows(len(df.index) - wks.row_count + col_names + last_idx_adjust)
    if len(df.columns) + row_names + last_col_adjust  > wks.col_count:
        wks.add_cols(len(df.columns) - wks.col_count + row_names + last_col_adjust)

    # Define first cell for rows and columns
    first_col = re.split('(\d+)',(wks.get_addr_int(1, start_col_int + 1)))[0].upper() if row_names else start_col
    first_row = str(start_row_int + 1) if col_names else start_row

    # Addition of col names
    if col_names:
        cell_list = wks.range('%s%s:%s%s' % (first_col, start_row, last_col, start_row))
        for idx, cell in enumerate(cell_list):
            cell.value = df.columns.values[idx]
        wks.update_cells(cell_list)

    # Addition of row names
    if row_names:
        cell_list = wks.range('%s%s:%s%d' % (start_col, first_row, start_col, last_idx))
        for idx, cell in enumerate(cell_list):
            cell.value = df.index[idx]
        wks.update_cells(cell_list)

    # Addition of cell values
    cell_list = wks.range('%s%s:%s%d' % (
        first_col, first_row, last_col, last_idx))
    for j, idx in enumerate(df.index):
        for i, col in enumerate(df.columns.values):
            cell_list[i + j * len(df.columns.values)].value = df[col][idx]
    for cells in grouper(chunk_size, cell_list):
        wks.update_cells(list(cells))

    return wks

In [ ]:
upload(df=final_df,wks=wks,credentials=credentials)